In [1]:
import matplotlib
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os


import seaborn as sns
from matplotlib import rcParams

#绘图前的基本设置

myfont = fm.FontProperties(fname=r'C:\Windows\Fonts\Times New Roman.tiff') # 设置字体

# 设置matplotlib正常显示中文和负号
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
matplotlib.rcParams['font.size'] = 12

# matplotlib.rcParams['font.sans-serif']=['NSimSun']   # 用黑体显示中文
# matplotlib.rcParams['axes.unicode_minus']=False     # 正常显示负号
# matplotlib.rcParams['font.size'] = 0.3
# matplotlib.rcParams['font.family'] = 'sans-serif'  
matplotlib.rcParams['font.sans-serif'] = 'Times New Roman'  # NSimSun 新宋体字体   宋体：SimSuncss

%matplotlib inline  
#内嵌绘图，并且可以省略掉plt.show()这一步
# os.chdir('D:\\BaiduSyncdisk\\EHF-文献-数据-文章\\EHF\\EHF\\figure-统计\\mean') #笔记本电脑
os.chdir(r'E:\BaiduSyncdisk\热浪博士论文\计算表格\Chapter4')     #台式机

cmap =list(["steelblue", "yellowgreen","orange","lightcoral","red", "green","lightskyblue"])

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Load the data
data_path = 'Historical_FuturePopulation_Exposure_by_year_province_exposure_pop_hwf.csv'
df = pd.read_csv(data_path)

# Ensure 'Year' is treated as a numeric column
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Define the periods for analysis with explicit scenario and timeframe filtering
periods = {
    'Historical (2001-2020)': ('TM_historical', 2001, 2020),
    'SSP2-4.5 (2041-2070)': ('TM_SSP245', 2041, 2070),
    'SSP2-4.5 (2071-2100)': ('TM_SSP245', 2071, 2100),
    'SSP5-8.5 (2041-2070)': ('TM_SSP585', 2041, 2070),
    'SSP5-8.5 (2071-2100)': ('TM_SSP585', 2071, 2100)
}

# Initialize a DataFrame to store results
results_df = pd.DataFrame()

# Function to calculate metrics
def calculate_metrics(data, period_label):
    metrics = {}
    years = data['Year'].values
    for column in ['Population', 'HWF', 'Population_Exposure']:
        mean = data[column].mean()
        std = data[column].std()
        slope, _, _, _, _ = linregress(years, data[column])
        
        # Store the results in a dictionary, rounding to two decimal places
        metrics[f'{column} Mean'] = round(mean, 2)
        metrics[f'{column} StdDev'] = round(std, 2)
        metrics[f'{column} Annual Growth'] = round(slope, 4)  # Annual growth rate

    return pd.DataFrame(metrics, index=[period_label])

# Loop through each period and calculate metrics for each province
for period_label, (scenario, start_year, end_year) in periods.items():
    period_data = df[(df['Year'] >= start_year) & (df['Year'] <= end_year) & (df['Variable'] == scenario)]
    for province in period_data['Province'].unique():
        province_data = period_data[period_data['Province'] == province]
        province_results = calculate_metrics(province_data, period_label)
        province_results['Province'] = province
        results_df = pd.concat([results_df, province_results])

# Set the index for the results DataFrame
results_df.set_index(['Province'], append=True, inplace=True)

# Round the entire DataFrame to two decimal places before saving
results_df = results_df.round(2)

# Save the results to an Excel file with more descriptive column names
output_path = 'PD-04-02-表格-分省份-热浪暴露总数时序变化.xlsx'
results_df.to_excel(output_path)

print(f"Results saved to {output_path}")


Results saved to Analysis_Results_Detailed_Periods.xlsx
